In [ ]:
import pandas as pd
df = pd.read_csv('research candidate sets.csv')
print(df.shape)
print(df.dtypes)

(14, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    import ast
    authorList = ast.literal_eval(row.authorList)
    
    val = 4
    
    test = "trainableoff"
    
    level = "word"
    
    iterations = 30
    
    dropout = 0.4
    
    samples = 300
    
    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(val)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(200), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))
    
    print("Current test: %s" % (str(printstate)))
    
    import UpdateDB as db
    
    case = db.checkLSTM(doc_id = doc_id, candidate = val, dimensions = 200, samples = 300, 
                        iterations = iterations, dropout = 0.2, test = test)
    
    if case == False:
        
        print("Running: %12s" % (str(printstate)))
        
        import StyloNeural as Stylo
        (labels_index, predYList, predY, history) = Stylo.getResults(authorList = authorList[:(val + 1)], 
                                                                     doc_id = doc_id, 
                                                                     level = level, 
                                                                     glove = '../../glove/',
                                                                     nb_epoch = iterations,
                                                                     samples = samples, 
                                                                     dimensions = 200,
                                                                     dropout = dropout, 
                                                                     batch_size = 30
                                                                    )
        
        import UpdateDB as db
        db.updateresultLSTM(doc_id = doc_id, candidate = val, dimensions = 200, samples = samples, 
                            iterations = iterations, dropout = dropout, accuracy = predY[0], test = test)
        
        from IPython.display import clear_output
        clear_output()
    
    else:
        print("Skipped: %12s" % (str(printstate)))
        

Current test: doc_id = 261, candidate = 4, dimensions = 200, samples = 300, 
iterations = 30, dropout = 0.4, test = trainableoff
Execution completed
Running: doc_id = 261, candidate = 4, dimensions = 200, samples = 300, 
iterations = 30, dropout = 0.4, test = trainableoff
Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 97
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (7158, 2)
Author:    31  Size:  1580
Author:    33  Size:  2573
Author:   104  Size:  2573
Author:   122  Size:  2573
Author:   161  Size:  2573
Min: 305
Max: 2573
Authors [31, 33, 104, 122, 161].
Found 1500 texts.
Found 1500 labels.
Found 43509 unique tokens.
Shape of data tensor: (1500, 1000)
Shape of label tensor: (1500, 5)
Done compiling.
Train on 1200 samples, validate on 300 samples
Epoch 1/30
1200/1200 [==============================] - 39s - loss: 1.4297 - acc: 0.3783 - val_loss: 1.2972 - val_acc: 0.4300

In [ ]:
# import matplotlib.pyplot as plt
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
%tb